## ERDDAP Moonflower Daily Plots

In [12]:
from erddapy import ERDDAP
import pandas as pd
import datetime

# for secondary/derived parameters
from metpy.units import units
import metpy.calc as mpcalc

In [13]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean

from matplotlib.dates import YearLocator, WeekdayLocator, MonthLocator, DayLocator, HourLocator, DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.ticker import NullFormatter  # useful for `logit` scale

In [14]:
### specify primary bulk figure parameters
fontsize = 20
labelsize = 16
#plotstyle = 'seaborn'
max_xticks = 10
plt.style.use('seaborn-ticks')
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['ps.fonttype'] = 42 #truetype/type2 fonts instead of type3
mpl.rcParams['pdf.fonttype'] = 42 #truetype/type2 fonts instead of type3
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['xtick.major.size'] = 4
mpl.rcParams['xtick.minor.size'] = 1
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 4
mpl.rcParams['ytick.minor.size'] = 1
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['xtick.color'] = 'black'

In [15]:
server_url = 'http://raspberrypi.local:8080/erddap'
server_url = 'http://192.168.2.15:8080/erddap'

In [16]:
(datetime.datetime.utcnow()-datetime.timedelta(days=14)).strftime('%Y-%m-%dT00:00:00Z')

'2020-07-31T00:00:00Z'

In [17]:
alldatasets=['tempest_moonflower_wx',
             'channel_1027974_thingspeak',
            'channel_1037066_thingspeak',
            'channel_1047747_thingspeak',
            'channel_843357_thingspeak',
            'channel_rpi']

constraints = {
    'time>=': (datetime.datetime.utcnow()-datetime.timedelta(days=14)).strftime('%Y-%m-%dT00:00:00Z'),
    'time<=': datetime.datetime.utcnow().strftime('%Y-%m-%dT00:00:00Z'),
}
df_all = {}

for dataset_id in alldatasets:
    try:
        d = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        d.dataset_id=dataset_id
        d.constraints=constraints
        
    except HTTPError:
        print('Failed to generate url {}'.format(dataset_id))

    try:
        df_m = d.to_pandas(
                    index_col='time (UTC)',
                    parse_dates=True,
                    skiprows=(1,)  # units information can be dropped.
                    )
        df_m.sort_index(inplace=True)
        df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

    except:
        print(f"something failed in data download {dataset_id}")
        pass

    #stats are all utc driven - but we really want local daily values
    if not dataset_id in ['channel_rpi']:
        df_m=df_m.tz_convert('US/Pacific')
    else:
        df_m=df_m.tz_convert(None).tz_localize(tz='US/Pacific')
    df_all.update({dataset_id:df_m})
    # calculations of various parameters... metpy?
    # HDD/CDD, dewpointTemp

something failed in data download tempest_moonflower_wx
something failed in data download channel_1027974_thingspeak
something failed in data download channel_1037066_thingspeak
something failed in data download channel_1047747_thingspeak
something failed in data download channel_843357_thingspeak
something failed in data download channel_rpi


In [18]:
import altair as alt
### own colormap
import palettable

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [19]:
dfsub=None
dfsub = pd.concat([df_all['tempest_moonflower_wx'].rename(columns={'temperature': 'Tempest_Temperature', 'humidity': 'Tempest_Humidity'}),
                   df_all['channel_1027974_thingspeak'].rename(columns={'temperature': 'TysonsRoomTemp'}),
                   df_all['channel_843357_thingspeak'].rename(columns={'Barotemperature': 'ShopTemp'}),                   
                   df_all['channel_1037066_thingspeak'].rename(columns={'temperature': 'CellarTemp', 'RH_Percent': 'CellarHumidity'}),
                   df_all['channel_1047747_thingspeak'].rename(columns={'temperature_internal': 'DuckBarnTemp_Internal',
                                                                        'temperature_external': 'DuckBarnTemp_External',
                                                                        'RH_Percent': 'Duckbarn_Humidity'}),
                   df_all['channel_rpi'].rename(columns={'temperature': 'OfficeRoomTemp', 'humidity': 'OfficeRoom_Humidity'})
                  ]).resample('15T').mean()


In [20]:
selector = alt.selection_single(
    fields=['key'], 
    empty='all',
    bind='legend'
)

area1 = alt.Chart(dfsub.reset_index()).transform_fold(
    ['DuckBarnTemp_Internal','DuckBarnTemp_External','TysonsRoomTemp','duckTd','ShopTemp','CellarTemp','OfficeRoomTemp','Tempest_Temperature']
).mark_line(clip=True
).encode(
    alt.X('time (UTC):T'),
    alt.Y('value:Q'),
    alt.Color('key:N'),
    opacity=alt.condition(selector, alt.value(1), alt.value(0))
).add_selection(
    selector
).properties(
    width=750,
    height=150
).interactive()

area1

alt.Chart(...)

In [21]:
f1 = alt.Chart(dfsub.reset_index()).mark_rect().encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - inside duck barn'),
    alt.Y('monthdate(time (UTC)):O', title='date'),
    alt.Color('DuckBarnTemp_Internal:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','DuckBarnTemp_Internal:Q']
).properties(
    width=900,
    height=200
)

f2 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - outside duck barn'),
    alt.Color('DuckBarnTemp_External:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','DuckBarnTemp_External:Q']
)


f3 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - Cellar'),
    alt.Color('CellarTemp:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','CellarTemp:Q']
)

f4 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - Shop'),
    alt.Color('ShopTemp:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','ShopTemp:Q']
)

f5 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - TysonsRoom'),
    alt.Color('TysonsRoomTemp:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','TysonsRoomTemp:Q']
)

f6 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - Tempest Temp'),
    alt.Color('Tempest_Temperature:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','Tempest_Temperature:Q']
)

f7 = f1.encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day - Office Temp'),
    alt.Color('OfficeRoomTemp:Q', title='temperature (C)', scale=alt.Scale(range=palettable.cmocean.sequential.Thermal_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','OfficeRoomTemp:Q']
)

(f1 & f2 & f3 & f4 & f5 & f6 & f7)

alt.VConcatChart(...)

In [22]:
f3 = alt.Chart(dfsub.reset_index()).mark_rect().encode(
    alt.X('hoursminutes(time (UTC)):O', title='hour of day'),
    alt.Y('monthdate(time (UTC)):O', title='date'),
    alt.Color('Duckbarn_Humidity:Q', title='humidity (%)', scale=alt.Scale(domain=(50,100),range=palettable.cmocean.sequential.Algae_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','Duckbarn_Humidity:Q']
).properties(
    width=900,
    height=200
)

f4 = f3.encode(
    alt.Color('CellarHumidity:Q', title='humidity (%)', scale=alt.Scale(domain=(50,100),range=palettable.cmocean.sequential.Algae_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','CellarHumidity:Q']
)

f5 = f3.encode(
    alt.Color('OfficeRoom_Humidity:Q', title='humidity (%)', scale=alt.Scale(domain=(50,100),range=palettable.cmocean.sequential.Algae_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','OfficeRoom_Humidity:Q']
)

f6 = f3.encode(
    alt.Color('Tempest_Humidity:Q', title='humidity (%)', scale=alt.Scale(domain=(50,100),range=palettable.cmocean.sequential.Algae_20.hex_colors)),
    tooltip=['hoursminutes(time (UTC)):O','monthdate(time (UTC)):O','Tempest_Humidity:Q']
)
f3 & f4 & f5 & f6

alt.VConcatChart(...)